<a href="https://colab.research.google.com/github/ayaanzhaque/SuiSense/blob/master/notebooks/bert/bert_sole_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing relevant libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB

import torch

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer


import warnings
warnings.filterwarnings('ignore')

In [ ]:
features = np.loadtxt('features_real_data_for_model.csv')

In [ ]:
features.shape

(1895, 768)

In [ ]:
print(features)

[[ 0.19605026  0.20534156  0.24803329 ...  0.15781963  0.45455933
  -0.41783777]
 [-0.11692914 -0.39638266  0.09059574 ... -0.02261155  0.26243898
   0.33010307]
 [ 0.11361607 -0.04054473  0.23259798 ... -0.1438802   0.45649728
   0.47679883]
 ...
 [ 0.04628711  0.53464365 -0.19218408 ... -0.34011787  0.77088737
   0.60213029]
 [ 0.24700873  0.28108341  0.21017519 ... -0.1912898   0.26677379
  -0.13201511]
 [ 0.16641247  0.16862674  0.03572434 ... -0.15101409  0.51394463
   0.24030764]]


In [ ]:
df = pd.read_csv('/content/real_data_for_model.csv')
og_batch_1 = df[['selftext', 'is_suicide']]

#batch_1_start = og_batch_1.head(60)
#batch_1_end = og_batch_1.tail(60)
#test_batch_1 = pd.concat([batch_1_start, batch_1_end], ignore_index=True)
batch_1 = og_batch_1.rename(columns={'selftext': 0, 'is_suicide': 1})

labels = batch_1[1]
labels.head()


0    0
1    0
2    0
3    0
4    0
Name: 1, dtype: int64

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.25, random_state=42, stratify=labels)

In [ ]:
train_features = np.asarray(train_features)
train_labels = np.asarray(train_labels)
test_features = np.asarray(test_features)
test_labels = np.asarray(test_labels)

In [ ]:
# parameters = {'C': np.linspace(0.0001, 100, 20)}
# grid_search = GridSearchCV(LogisticRegression(), parameters)
# grid_search.fit(train_features, train_labels)

# print('best parameters: ', grid_search.best_params_)
# print('best scrores: ', grid_search.best_score_)

In [ ]:
#!pip install tensorflow==2.0.0

In [ ]:
features.shape

(1895, 768)

In [ ]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D, Input

model3 = Sequential()

model3.add(Input(shape=(768,), name="input_word_ids"))

model3.add(Dense(1, activation='sigmoid'))

model3.add(Dense(10, activation='relu'))
model3.add(Dense(1, activation='sigmoid'))

model3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model3.fit(train_features, train_labels, batch_size=32, epochs=19, verbose=1, validation_data=(test_features, test_labels))

Train on 1421 samples, validate on 474 samples
Epoch 1/19
1421/1421 [==============================] - 1s 763us/sample - loss: 0.6899 - accuracy: 0.5222 - val_loss: 0.6850 - val_accuracy: 0.5274
Epoch 2/19
1421/1421 [==============================] - 0s 122us/sample - loss: 0.6777 - accuracy: 0.5327 - val_loss: 0.6797 - val_accuracy: 0.5211
Epoch 3/19
1421/1421 [==============================] - 0s 121us/sample - loss: 0.6705 - accuracy: 0.5588 - val_loss: 0.6759 - val_accuracy: 0.6118
Epoch 4/19
1421/1421 [==============================] - 0s 128us/sample - loss: 0.6634 - accuracy: 0.5890 - val_loss: 0.6700 - val_accuracy: 0.5992
Epoch 5/19
1421/1421 [==============================] - 0s 109us/sample - loss: 0.6547 - accuracy: 0.6115 - val_loss: 0.6659 - val_accuracy: 0.5949
Epoch 6/19
1421/1421 [==============================] - 0s 111us/sample - loss: 0.6496 - accuracy: 0.6094 - val_loss: 0.6590 - val_accuracy: 0.6477
Epoch 7/19
1421/1421 [==============================] - 0s 115us/

In [ ]:
model3.save("final_regular_model.h5")

How good is this score? What can we compare it against? Let's first look at a dummy classifier:

In [ ]:
final_model = keras.models.load_model('final_regular_model.h5')

In [ ]:


prediction = final_model.predict(features)
print(prediction)

[[0.32071516]
 [0.33665255]
 [0.3259926 ]
 ...
 [0.79324293]
 [0.6191386 ]
 [0.7364468 ]]


In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.507 (+/- 0.02)
